In [2]:
#本段用于修改客户明细账格式，如果已经修改好了就直接运行下段，运行前需要将前18个月的数据都放到一张表中（只有客户明细账导出后的格式不变的情况下才能使用，变了要修改本段内容）
#18个月是因为本程序需要本月即将到期金额的账单期往前推12个月，比如2024年12月计算款期为‘每月20号止,月结120天’的超期，其本月即将到期金额的账单期为2024.7.21-8.20，需要推到2023.7.21-8.20，所以如果款期超过5个月还要再增加数据
#放在这是为了避免因下段的输入错误而重新运行时，因客户明细账已经改好了格式而报错
#操作为：删9、11、13列、1、2、3、5行，修改'本期借方':'借方原币','本期贷方':'贷方原币','期末余额':'期末原币'，按日期升序排序
import pandas as pd
khmx_dz=input('请输入客户明细账文件地址（不带引号，请确认已关闭该文件）：')
khmx_sheet=input('请输入客户明细账所在工作表名：')
khmx=pd.read_excel(khmx_dz,sheet_name=khmx_sheet,header=3)
khmx.drop(khmx.columns[[8,10,12]],axis=1,inplace=True)
khmx.drop([0],axis=0,inplace=True)
khmx.rename(columns={'本期借方':'借方原币','本期贷方':'贷方原币','期末余额':'期末原币'},inplace=True)
khmx=khmx.dropna(subset=['借方原币','贷方原币'],how='all')
khmx['辅助列']=khmx.reset_index().index                     #用代码排序时日期一样的订单顺序会乱，会影响下面最后一个订单的读取，所以加个辅助列存储原始顺序，将其当做次要关键字
khmx=khmx.sort_values(by=['日期', '辅助列'], ascending=[True, True])   #按日期升序排序，为了方便下面取得最后一个订单的期末原币（因为超期表里有期末余额是本币，计算需要原币）
khmx=khmx.drop(columns=['辅助列'])
khmx.to_excel(khmx_dz,sheet_name=khmx_sheet,index=False)    #将调整格式后的数据保存到原文件地址，想保存到别的位置就修改代码
print('运行完成')

请输入客户明细账文件地址（不带引号）： C:\Users\荣耀\Desktop\客户明细账 （全）.xlsx
请输入客户明细账所在工作表名： 客户明细账


运行完成


In [4]:
#交互版
def overdue_calculate(cqb,cqb_dict,client,latter,former,row,overdue_header,overdue_index,sell_money,due_money,period):
    #cqb:超期表文件,cqb_dict:超期表字典列表,client:单客户列表字典,latter,former:这段时期的开始和截止日期row:行号,overdue_header,overdue_index:
    #各时期计入的列名列表及其索引，sell_money：当期销售金额，due_money：当期总应收金额，period：判断是否需要调用输出本月最后一天的自定义函数
    for j in client:
        if former<j['日期']<=latter:
            sell_money[overdue_index]+=round(float(j['借方原币']),2)
    if due_money[overdue_index]>0:     #当应付余额小于等于0，应该在对应地方记0，但因为前面已经把所有空值都填充成了0，这里就可以直接退出函数了，所以直接取反
        if overdue_index==12 and sell_money[overdue_index]<due_money[overdue_index]:      #超期一年以上就不往下算了
            cqb_dict[row][overdue_header[overdue_index]]=round(due_money[overdue_index],2)       #超期11-12月的部分
            cqb.at[row,overdue_header[overdue_index]]=round(due_money[overdue_index],2)*exchange_rate[i['币种']]
            cqb_dict[row][overdue_header[overdue_index+1]]=round(due_money[overdue_index]-sell_money[overdue_index],2)       #超期一年以上的部分
            cqb.at[row,overdue_header[overdue_index+1]]=round(due_money[overdue_index]-sell_money[overdue_index],2)

        
        elif sell_money[overdue_index]>=due_money[overdue_index]:
            cqb_dict[row][overdue_header[overdue_index]]=round(due_money[overdue_index],2)       #列表存储的是原币，用于后面加批注时获取原币金额
            cqb.at[row,overdue_header[overdue_index]]=round(due_money[overdue_index],2)*exchange_rate[i['币种']]     #Excel存储的是本币
            #无超期情况下将剩余应付金额计入即将到期金额，列表和Excel都更新，并跳出循环与自定义函数
        elif sell_money[overdue_index]<due_money[overdue_index]:   #本期销售额<本期总应收额
            if sell_money[overdue_index]<0:
                sell_money[overdue_index]=0
            cqb_dict[row][overdue_header[overdue_index]]=round(sell_money[overdue_index],2)
            cqb.at[row,overdue_header[overdue_index]]=round(sell_money[overdue_index],2)*exchange_rate[i['币种']]
            due_money[overdue_index+1]=round(due_money[overdue_index]-sell_money[overdue_index],2)
            overdue_index+=1
            latter-=relativedelta(months=1)
            former-=relativedelta(months=1)
            if period[0]==30:
                latter=last_day(latter.year,latter.month)
                former=last_day(former.year,former.month)
            overdue_calculate(cqb,cqb_dict,client,latter,former,row,overdue_header,overdue_index,sell_money,due_money,period)
            #超期的情况下本时段销售额直接计入即将到期金额，列表和Excel都更新，并开启下一时间段的循环




def last_day(year,month):       #用于账单30号截止时取得每月最后一天
    date=pd.Timestamp(year=year, month=month, day=1)
    return (date+pd.offsets.MonthEnd(0)).date()




import pandas as pd
import openpyxl
from openpyxl.comments import Comment
import string
import datetime
import re
from collections import Counter
from dateutil.relativedelta import relativedelta

khmx_dz=input('请输入客户明细账文件地址（不带引号）：')      #输入地址之前需要把前18个月的数据放到一张表里，并确保表头格式名称格式正确
khmx_sheet=input('请输入客户明细账所在工作表名：')
khmx=pd.read_excel(khmx_dz,sheet_name=khmx_sheet)
khmx=khmx.fillna(0)                                      #将表中的空值填充成0
khmx_dict=khmx.to_dict(orient='records')
cqb_dz=input('请输入超期表文件地址（不带引号）：')
#输入地址之前需要将其表头的格式及名称调好(不限制顺序，到期超期合计那三个会随月份变化的记得改)，款项匹配好（并按要求格式改好）
cqb_sheet=input('请输入超期表所在工作表名：')
cqb=pd.read_excel(cqb_dz,sheet_name=cqb_sheet)
cqb=cqb.fillna(0)
cqb_dict=cqb.to_dict(orient='records')

#本段用于确认超期表中的公司有两个币种但在超期表只有一行的情况，以便操作者在运行程序之前确定是否在超期表添加行（如果不在超期表加行也不用删掉客户明细账中的多余数据）
dual_currency=[]
for i in cqb_dict:
    for j in khmx_dict:
        if i['客户']==j['客户'] and i['币种']!=j['币种']:
            dual_currency.append(i['客户'])
            break
count=Counter(dual_currency)
dual_currency=[a for a,cnt in count.items() if cnt==1]
if dual_currency!=[]:
    print('以下客户多币种核算且未在超期表中列明，请确定是否需要在超期表中加行\n'+'\n'.join(dual_currency))
    wait=input('若无需修改超期表或已修改好，请按回车')
    cqb=pd.read_excel(cqb_dz,sheet_name=cqb_sheet)
    cqb=cqb.fillna(0)
    cqb_dict=cqb.to_dict(orient='records')
    
month_today=(input('请输入查询年月，格式如2024-01：'))
today=datetime.datetime.strptime(month_today,'%Y-%m').date()
exchange_rate={}                                       #存储币种汇率的字典，根据实际情况增加
exchange_rate['美元']=float(input('请输入美元汇率：'))
exchange_rate['欧元']=float(input('请输入欧元汇率：'))
exchange_rate['人民币']=1
columns_key=list(cqb.columns)                   #存储所有列名的列表
letters=string.ascii_uppercase                   #这里只存储了A-Z,没有后面的AA、AB等，现在正好用到Z，后面要再加行的话要改一下这里
cqb_header_dict=dict(zip(columns_key,letters))    #存储了每一个列名对应的字母序号，用于后面加批注的时候确认列号（openpyxl不能直接用列名修改）
for i in columns_key:
    if i[-5:]=='月份到期款':
        current_due_name=i                  #因为每个月的这三个列名都会变，所以赋值给变量后面好引用，这三个存储的都是列名，分别代表本月到期、上月总超期、本月应收
    elif i[-6:]=='月份超期金额':
        all_overdue_name=i
    elif i[-7:]=='月份应收款合计':
        all_receivable_name=i
overdue_header=[current_due_name,'超期1个月以内','超期1-2月','超期2-3月','超期3-4月','超期4-5月','超期5-6月','超期6-7月','超期7-8月','超期8-9月','超期9-10月','超期10-11月','超期11-12月','超期1年以上',all_overdue_name,all_receivable_name,'超期3-6月','超期6月-1年']
#用于在自定义函数更新数据时获得对应列名（只用到了前14个），以及在加批注时遍历

row=-1
for i in cqb_dict:
    row+=1                            #用于后面填数据的时候确认行号
    if i['款期'].find('（')!=-1:      #款期格式要求：优先取括号中的数，只能是 款到发货 或 每月X号止，月结X天 （两个数的值和顺序对就行，文字没有格式要求）
        i['款期']=i['款期'][i['款期'].find('（')+1:-1]
    elif i['款期'].find('(')!=-1:
        i['款期']=i['款期'][i['款期'].find('(')+1:-1]
    if i['款期']=='款到发货' and i['币种']=='人民币':     #内贸算每月20号止，月结0天，外贸算每月30号止，月结0天，有特殊情况的就加括号（其实这样算内贸外贸在这里整除后都是0个月，不分开写也行？）
        period=[20,0]                                   #period存储款期，第一个数是截止日，第二个数是月结天数
    elif i['款期']=='款到发货' and i['币种']!='人民币':
        period=[30,0]
    else:
        period=re.findall(r'\d+',i['款期'])
        period=[int(j) for j in period]
    if today.month==2 and period[0]==30:
        latter=datetime.datetime.strptime(month_today+'-'+str(period[0]),'%Y-%m-%d').date()-relativedelta(months=(period[0]+period[1]-1)//30)
    else:
        latter=datetime.datetime.strptime(month_today+'-'+str(period[0]),'%Y-%m-%d').date()-relativedelta(months=(period[0]+period[1]-1)//30)
    #latter指本月到期金额截止日期（后面进入自定义函数时会变成计算时段的最后日期）
    former=latter-relativedelta(months=1)           #former指超期金额截止日期（后面进入自定义函数时会变成计算时段的开始日期）
    if period[0]==30:                                     #账单30号截止时取每月最后一天
        latter=last_day(latter.year,latter.month)
        former=last_day(former.year,former.month)
    
    client=[a for a in khmx_dict if a['客户']==i['客户'] and a['币种']==i['币种']]    #client指特定客户与币种的明细账字典列表
    sell=0
    for j in range(len(client)):
        if type(client[j]['日期'])!=datetime.date:
            client[j]['日期']=datetime.datetime.strptime(client[j]['日期'],'%Y-%m-%d').date()  #将这一客户字典列表日期都变成日期类型
        if latter<client[j]['日期']<=today:
            sell+=float(client[j]['借方原币'])                   #即将到期金额截止日期后的销售额
    if len(client)==0:
        print('未查询到',i['客户'],'相关信息')
        continue
    begin_money=0
    overdue_index=0             #在自定义函数做overdue_header的索引
    sell_money=[0]*14              #记录每期的销售金额、当期总应收余额
    due_money=[0]*14
    due_money[0]=round(client[-1]['期末原币']-sell,2)
    overdue_calculate(cqb,cqb_dict,client,latter,former,row,overdue_header,overdue_index,sell_money,due_money,period)      #计算超期
    
    cqb_dict[row]['超期3-6月_str']=''          #计算其他统计数据（后面统计上月总超期需要超期3-6月、超期6月-1年的数值数据，所以各多加一个用作批注的键）
    cqb_dict[row]['超期6月-1年_str']=''
    for j in range(4,13):
        if j in range(4,7):
            key='超期3-6月'
        else:
            key='超期6月-1年'
        if cqb_dict[row][overdue_header[j]]!=0:
            cqb_dict[row][key+'_str']+=str(round(cqb_dict[row][overdue_header[j]],2))+cqb_dict[row]['币种']+'，'+overdue_header[j]+'\n'
        cqb_dict[row][key]+=round(cqb_dict[row][overdue_header[j]],2)
        cqb.loc[row,key]+=cqb.loc[row,overdue_header[j]]
    all_overdue_list=['超期1个月以内','超期1-2月','超期2-3月','超期3-6月','超期6月-1年','超期1年以上']
    for j in all_overdue_list:
        cqb_dict[row][all_overdue_name]+=round(cqb_dict[row][j],2)
        cqb.loc[row,all_overdue_name]+=cqb.loc[row,j]
    cqb_dict[row][all_receivable_name]=round(cqb_dict[row][current_due_name]+cqb_dict[row][all_overdue_name],2)
    cqb.loc[row,all_receivable_name]=cqb.loc[row,current_due_name]+cqb.loc[row,all_overdue_name]
finished_cqb_dz=input('请输入存储地址（不带引号，请确认已经关闭该文件）：')
finished_cqb_sheet=input('请输入存储工作表：')
cqb.to_excel(finished_cqb_dz,index=False)       #index=False指不保存索引；批注只能用openpyxl不能用pandas所以要先保存；地址记得改

cqb_file=openpyxl.load_workbook(finished_cqb_dz)
cqb1=cqb_file[finished_cqb_sheet]
commenter=input('请输入批注者：')
row=1
for i in cqb_dict:
    row+=1
    if i['币种']!='人民币':                  #除超期3-6月、超期6月-1年的其他数据中外币批注的处理
        for j in overdue_header[:16]:
            if cqb_dict[row-2][j]!=0:
                cqb1[cqb_header_dict[j]+str(row)].comment=Comment(str(cqb_dict[row-2][j])+i['币种'],commenter)
    for j in overdue_header[16:]:           #所有超期3-6月、超期6月-1年数据批注的处理（不同币种与不同月份在上面已经处理好了）
        if cqb_dict[row-2][j]!=0:
            cqb1[cqb_header_dict[j]+str(row)].comment=Comment(cqb_dict[row-2][j+'_str'],commenter)
cqb_file.save(finished_cqb_dz)
print('运行完成')
#完成后可以设置一下单元格格式

请输入客户明细账文件地址（不带引号）： C:\Users\荣耀\Desktop\客户明细账 （全）.xlsx
请输入客户明细账所在工作表名： 客户明细账
请输入超期表文件地址（不带引号）： C:\Users\荣耀\Desktop\超期表.xlsx
请输入超期表所在工作表名： 测试


以下客户多币种核算且未在超期表中列明，请确定是否需要在超期表中加行
THAI KK INDUSTRY CO.,LTD.
Maxthetic International Co.,Ltd.
SYMBIO, INC
Victoria Srl
BUDAVAL


若无需修改超期表或已修改好，请按回车 
请输入查询年月，格式如2024-01： 2024-12
请输入美元汇率： 7.1865
请输入欧元汇率： 7.5907


未查询到 浙江永冠翊威数码科技有限公司 相关信息
未查询到 广州九桥合成纸业有限公司 相关信息
未查询到 Wiley Eco Print Industrial Co., Ltd. 相关信息


请输入存储地址（不带引号，请确认已经关闭该文件）： C:\Users\荣耀\Desktop\超期表(测试).xlsx
请输入存储工作表： Sheet1
请输入批注者： 1


运行完成
